In [11]:
import json

In [12]:
import geopandas as gpd
import pandas as pd
import requests
from unidecode import unidecode

In [13]:
url = "https://pxweb.stat.si:443/SiStatData/api/v1/en/Data/0701024S.px"

In [44]:
data = {
    "query": [
        {
            "code": "OBČINE",
            "selection": {
                "filter": "item",
                "values": [
                    "0",
                    "001",
                    "213",
                    "195",
                    "002",
                    "148",
                    "149",
                    "003",
                    "150",
                    "004",
                    "005",
                    "006",
                    "151",
                    "007",
                    "008",
                    "009",
                    "152",
                    "011",
                    "012",
                    "013",
                    "014",
                    "153",
                    "196",
                    "015",
                    "016",
                    "017",
                    "018",
                    "019",
                    "154",
                    "020",
                    "155",
                    "021",
                    "156",
                    "022",
                    "157",
                    "023",
                    "024",
                    "025",
                    "026",
                    "027",
                    "028",
                    "207",
                    "029",
                    "030",
                    "031",
                    "158",
                    "032",
                    "159",
                    "160",
                    "161",
                    "162",
                    "034",
                    "035",
                    "036",
                    "037",
                    "038",
                    "039",
                    "040",
                    "041",
                    "163",
                    "042",
                    "043",
                    "044",
                    "045",
                    "046",
                    "047",
                    "048",
                    "049",
                    "164",
                    "050",
                    "197",
                    "165",
                    "051",
                    "052",
                    "053",
                    "166",
                    "054",
                    "055",
                    "056",
                    "057",
                    "058",
                    "059",
                    "060",
                    "061",
                    "062",
                    "063",
                    "208",
                    "064",
                    "065",
                    "066",
                    "167",
                    "067",
                    "068",
                    "069",
                    "198",
                    "070",
                    "168",
                    "071",
                    "072",
                    "073",
                    "074",
                    "169",
                    "075",
                    "212",
                    "170",
                    "076",
                    "199",
                    "077",
                    "078",
                    "079",
                    "080",
                    "081",
                    "082",
                    "083",
                    "084",
                    "085",
                    "086",
                    "171",
                    "087",
                    "088",
                    "089",
                    "090",
                    "091",
                    "092",
                    "172",
                    "093",
                    "200",
                    "173",
                    "094",
                    "174",
                    "095",
                    "175",
                    "096",
                    "097",
                    "098",
                    "099",
                    "100",
                    "101",
                    "102",
                    "103",
                    "176",
                    "209",
                    "201",
                    "104",
                    "177",
                    "106",
                    "105",
                    "107",
                    "108",
                    "178",
                    "109",
                    "110",
                    "111",
                    "112",
                    "113",
                    "114",
                    "179",
                    "180",
                    "202",
                    "115",
                    "203",
                    "181",
                    "204",
                    "182",
                    "116",
                    "210",
                    "205",
                    "033",
                    "183",
                    "117",
                    "118",
                    "119",
                    "120",
                    "211",
                    "121",
                    "122",
                    "123",
                    "124",
                    "206",
                    "125",
                    "194",
                    "126",
                    "127",
                    "184",
                    "010",
                    "128",
                    "129",
                    "130",
                    "185",
                    "186",
                    "131",
                    "132",
                    "133",
                    "187",
                    "134",
                    "188",
                    "135",
                    "136",
                    "137",
                    "138",
                    "139",
                    "189",
                    "140",
                    "141",
                    "142",
                    "143",
                    "144",
                    "190",
                    "146",
                    "191",
                    "147",
                    "192",
                    "193",
                ],
            },
        },
        {
            "code": "KAZALNIK",
            "selection": {"filter": "item", "values": ["PLA-MESEC-NETO"]},
        },
        {"code": "LETO", "selection": {"filter": "item", "values": ["2024"]}},
    ],
    "response": {"format": "json-stat"},
}

In [45]:
r = requests.post(url, data=json.dumps(data))

In [46]:
json_data = r.json()
indexes = json_data["dataset"]["dimension"]["OBČINE"]["category"]["index"]
labels = json_data["dataset"]["dimension"]["OBČINE"]["category"]["label"]
data = json_data["dataset"]["value"]

In [47]:
data_list = [{"name": labels[key], "NET": data[indx]} for key, indx in indexes.items()]

In [48]:
income_df = pd.DataFrame(data_list)
income_df["name"] = income_df["name"].str.split("/").str[0].apply(unidecode)
income_df["name"] = income_df["name"].str.replace(r"\s*-\s*", "-", regex=True)

## Open Geojson

In [49]:
slovenian_muni_gdf = gpd.read_file("data/si.json")
slovenian_muni_gdf = slovenian_muni_gdf[["name", "geometry"]]
slovenian_muni_gdf["name"] = slovenian_muni_gdf["name"].apply(unidecode)

In [50]:
to_replace = {
    "Kostanjevica": "Kostanjevica na Krki",
    "Nova Goriska": "Nova Gorica",
    "Novo Mesto": "Novo mesto",
    "Sentjur pri Celju": "Sentjur",
    "Smartno in Litiji": "Smartno pri Litiji",
    "Sveta Trojica v Slovenskih goricah": "Sveta Trojica v Slov. goricah",
    "Sveti Andraz v Slovenskih Goricah": "Sveti Andraz v Slov. goricah",
    "Sveti Jurij": "Sveti Jurij ob Scavnici",
    "Sveti Jurij v Slovenskih goricah": "Sveti Jurij v Slov. goricah",
}

In [51]:
slovenian_muni_gdf = slovenian_muni_gdf.replace(to_replace)

## Merge and Output

In [52]:
slovenian_muni_income_gdf = slovenian_muni_gdf.merge(income_df, on="name", how="left")

In [53]:
slovenian_muni_income_gdf[slovenian_muni_income_gdf["NET"].isna()]

,name,geometry,NET
62,Osilnica,"POLYGON ((14.72368 45.59698, 14.73009 45.57577...",NaN


In [43]:
slovenian_muni_income_gdf[slovenian_muni_income_gdf["NET"].isna()]["name"] == income_df[
    income_df["name"] == "Osilnica"
]["name"].iloc[0]

62    True
Name: name, dtype: bool

'Osilnica'

In [29]:
slovenian_muni_income_gdf = slovenian_muni_income_gdf.to_crs(3794)
slovenian_muni_income_gdf.to_file("data/slovenian_income.gpkg")